# Introduction



This notebook is a template for applying a PPO model in Webots continuous environment.

---
**NOTE**

To use this notebook, please first follow `UseGuide.md` to install the neccessary software and packages.

Import the PPO1 algorithm from stable baselines:

In [1]:
%%capture output 
# captures ALL output in cell to disable tensorflow warnings

from stable_baselines import PPO1

Import our gym:

In [2]:
import sys
sys.path.insert(0,'../backend')

import webotsgym as wg

# Setup World Parameters and Start the Webotsgym


You can setup the Webots environment parameters:

* `config.world_size`, setup the size of Webots environments for training. For example: `config.world_size = 8` will setup a square map of size 8x8 in Webots.
* `config.num_obstacles`, setup the number of obstacles. Each obstacle is a block of size 1x1.
* `config.sim_mode`, used to setup the speed of the simulation of Webots. 
`config.sim_mode = wg.config.SimSpeedMode.NORMAL`, run the simulation in the Real-Time mode.
`config.sim_mode = wg.config.SimSpeedMode.RUN`, run the simulation as fast as possible using all the available CPU power. 
`config.sim_mode = wg.config.SimSpeedMode.FAST`, run the simulation as fast as possible without the graphics rendering, hence the 3d window is black.
* `config.sim_step_every_x`, controls the interval between state/action in Webots time steps (32ms). The lower, the more frequent the actions. We recommend setting this to 5 for the continuous environment.

In [3]:
# set config
config = wg.WbtConfig()
config.world_size = 3
config.num_obstacles = 0
config.sim_mode = wg.config.SimSpeedMode.RUN
config.sim_step_every_x = 5

# start world
env = wg.WbtGym(config=config)

Accepting on Port:  10201


# Load a Model

Load a pretrained model to apply on the WbtGym. You must train your own model in `model_training_continuous_world.ipynb`. Due to the complexity of the continuous world we have not trained successful models for randomized worlds.

In [4]:
%%capture output 
# captures ALL output in cell to disable tensorflow warnings

model_name = "PPO_webots_continuous"
model = PPO1.load("model/continuous/{}".format(model_name))

ValueError: Error: the file model/continuous/PPO_webots_continuous could not be found

# Apply Model on a Random World

Randomly reset the start point, target point, and the obstacles in the environment and apply the loaded PPO model to this environment. Please set `step_limit` to limit the steps for the robot.

You can see how the robot goes to the target. If the robot reaches the target, it will print "Reach the target, congratulations!". If the robot doesn't reach the target in 2000 time steps, it will stop. Then you can rerun this cell to reset the environment. The `step_limit` must be set properly with regards to both the size/complexity of the environment and the frequency of the state/action pairs (controlled with the paramter `sim_step_every_x` in the config) to give the robot a suitable amount of time to reach the target.

In [5]:
obs = env.reset()
step_limit = 2000
for i in range(step_limit):
    (action, _states) = model.predict(obs)
    (obs, rewards, done, info) = env.step(action)

    if env.get_target_distance() < 0.1:
        print ('Congratulations! The robot reached the target.')
        break
    if i == step_limit - 1:
        print ('Reached step limit without reaching target.')

NameError: name 'model' is not defined